# Vistas basadas en clases (CBV)

In [ ]:
python manage.py startapp products

## products/views.py

In [ ]:
from django.shortcuts import render
from django.views.generic import View, ListView
from django.decorations.http import require_http_methods

from .models import Product, Post, User

# Create your views here.
def product_list_view(request):
    if request.method == "POST":
        print(request.POST)
    print(request.method == "POST")
    return render(request, "template", {})

class ProductHomeView(View):
    def get(self, request, *args, **kwargs):
        return render(request, "template", {})
    
    def post(self, request, *args, **ksargs):
        print(render.POST)
        return render(request, "template", {})
    
class ProductListView(ListView):
    queryset = Product.objects.all()

product_list_view = ProductListView.as_view()

class BlogPostListView(ListView):
    queryset = Post.objects.all()

class UsersPostsListView(ListView):
    queryset = User.objects.all()

# TemplateView

### src/templates/about.html

In [ ]:
<h1>Acerca del Ecommerce</h1>

### src/products/views.py

In [ ]:
from django.views.generic import View, ListView, TemplateView
from django.shortcuts import render

from .models import Product

class AboutUsView(View):
    def get(self, request):
        return render(request, "about.html", {})

class AboutUsTemplateView(TemplateView):
    def get(self, request):
        return render(request, "about.html", {})

### src/products/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path
from django.views.generic import TemplateView

from products import views

urlpatterns = [
    path("admin/", admin.site.urls),
    path("about/", TemplateView.as_view(template_name="about.html")),
    path("team/", TemplateView.as_view(template_name="team.html")),
]

### src/config/settings.py

In [ ]:
INSTALLED_APPS = [
    "products.apps.ProductsConfig", # <-------
    ...
]

### src/config/urls.py

In [ ]:
urlpatterns = [
    path("products/", include("products.urls")),
    ...
]

## RedirectView o Vista de Redireccionamiento

### src/products/urls.py

In [ ]:
drom django.views.generic import TemplateView, RedirectView # <-------

urlpatterns = [
    path("about-us/", RedirectView.as_view(url="/products/about/")), # <-------
    ...
]

## DetailView y ListView

### src/products/models.py

In [ ]:
from django.db import models

class Product(models.Model):
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)

### src/products/views.py

In [ ]:
from django.http import HttpResponseRedirect
from django.views.generic import DetailView, ListView
from django.shortcuts import render

from .models import Product

class ProductDetailView(DetailView):
    model = Product

class ProductListView(ListView):
    model = Product

### src/products/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path
from djnago.views.generic import TemplateView, RedirectView

from products.views import ProductListView, ProductDetailView

urlpatterns = [
    ...
    path("products/", ProductListView.as_view()),
    path("products/<int:pk>", ProductDetailView.as_view()),
    ...
]

### src/templates/products/product_list.html
### src/templates/products/product_details.html

### src/products/admin.py

In [ ]:
from django.contrib import admin

from .models import Product

# Register your models here.
admin.site.register(Product)

## Obtener la Data del contexto

In [ ]:
class ProductListView(ListView):
    ...

    def get_context_data(self, *args, **kwargs):
        context = super().get_context_data(*args, **kwargs)
        print(context)
        context["title"] = "Ecommerce"
        return context

### src/templates/products/product_list.html

In [ ]:
{{ title }}

{% for product in object_list %}
    <li> {{ object.title }} - {{ object.slug }} </li>
{% endfor %}

## Modelos Proxy

### src/products/models.py

In [ ]:
...

class DigitalProduct(Product):
    class Meta:
        proxy = True

### src/products/views.py

In [ ]:
...

class DigitalProductListView(ListView):
    model = DigitalProduct
    template_name = "products/product_list.html"

    def get_context_data(self, *args, **kwargs):
        context = super().get_context_data(*args, **kwargs)
        print(context)
        context["title"] = "Ecommerce"
        return context

### src/products/admin.py

In [ ]:
from django.contrib import admin

from .models import Product, DigitalProduct

# Register your models here.
admin.site.register(Product)
admin.site.register(DigitalProduct)

### src/products/mixins.py

In [ ]:
class TemplateTitleMixin(object):
    title = None

    def get_context_data(self, *args, **kwargs):
        context = super().get_context_data(*args, **kwargs)
        context["title"] = self.get_title()
        return context

    def get_title(self):
        return self.title

### src/products/views.py

In [ ]:
class ProductListView(ListView, TemplateTitleMixin):
    title = "Productos físicos"
    model = Product

class DigitalProductListView(ListView, TemplateTitleMixin):
    title = "Productos digitales"
    model = DigitalProduct
    template_name = "products/product_list.html"

### src/templates/products/product_list.html

In [ ]:
{% if title %}
    <h1>{{ title }}</h1>
{% endif %}

{% for product in object_list %}
    <li> {{ product.title }} - {{ product.slug }} </li>
{% endfor %}

## RedirectView basada en la instancia del Modelo

### src/products/views.py

In [ ]:
from django.http import HttpResponseRedirect
from django.views.generic import ListView, DetailView, RedirectView
from django.shortcuts import render, get_object_or_404

from .mixins import TemplateTitleMixin
from .models import Product, DigitalProduct

class ProductIDRedirectView(RedirectView):
    def get_redirect_url(self, *args, **kwargs):
        url_params = self.kwargs
        pk = url_params.get("pk")
        obj = get_object_or_404(Product, pk=pk)
        slug = obj.slug
        return f"/products/products/{slug}"

class ProductRedirectView(RedirectView):
        def get_redirect_url(self, *args, **kwargs):
            url_params = self.kwargs
            slug = url_params.get("slug")
            return f"/products/products/{slug}"

### src/products/models.py

In [ ]:
from django.db import models

class Product(models.Model):
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)

class DigitalProduct(Product):
    class Meta:
        proxy = True

### src/templates/products/product_detail.html

In [ ]:
{% if title %}
    <h1> {{ title }} </h1>
{% endif %}

 {{ object.pk }}
 {{ object.title }}
 {{ object.slug }}

### src/products/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path
from django.views.generic import TemplateView, RedirectView

from products.views import ProductListView, ProductDetailView, DigitalProductListView, ProductIDRedirectView, ProductRedirectView

path("products/<int:pk>", ProductDetailView.as_view()),
path("products/<slug:slug>", ProductDetailView.as_view()),
path("p/<int:pk>/", ProductIDredirectView.as_view()),
path("p/<slug:slug>/", ProductRedirectView.as_view()),

## Mixin para proteger con Login

### src/products/views.py

In [ ]:
...
from django.contrib.auth.mixins import LoginRequiredMixin # <-------

class ProtectedProductDetailView(LoginRequiredMixin, DetailView): # <-------
    model = Product
...

### src/products/urls.py

In [ ]:
...
    path("my-products/<slug:slug>", ProtectedProductDetailView.as_view()), # <-------

## Model Form

###  src/products/models.py

In [ ]:
from django.conf import settings # <-------
from django.db import models

User = settings.AUTH_USER_MODEL # <-------

class Product(models.Model):
    user = models.ForeignKey(User, blank=True, null=True, on_delete=models.SET_NULL) # <-------
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)

class DigitalProduct(Product):
    class Meta:
        proxy = True

### src/products/forms.py

In [ ]:
from django import forms

from .models import Product

class ProductModelForm(forms.Modelform):
    class Meta:
        model = Product
        fields = [
            "title",
            "slug",
        ]

## CreateView
### src/products/views.py

In [ ]:
from django.http import HttpResponseRedirect
from django.contrib.auth.mixins import LoginRequiredMixin
from django.views.generic import (
    ListView,
    DetailView,
    RedirectView,
    TemplateView,
    CreateView, # <-------
)
from django.shortcuts import render, get_object_or_404

from .forms import ProductModelForm # <-------
from .mixins import TemplateTitleMixin
from .models import Product, DigitalProduct

class ProtectedProductCreateView(LoginRequiredMixin, CreateView):
    form_class = ProductModelForm
    template_name = "forms.html"
    
    def form_valid(self, form):
        form.instance.user = self.request.user
        return super().form_valid(form)
    
    # def form_invalid(self, form):

### src/products/urls.py

In [ ]:
    path("my-products/create/", ProtectedProductCreateView.as_view()), # <-------

### src/templates/forms.html

<form method="POST" action=".">
    {% csrf_token %}
    {{ form.as_p }}

    <input type="submit" value="Guardar">
</form>

### src/products/models.py

In [ ]:
from django.conf import settings 
from django.db import models

User = settings.AUTH_USER_MODEL 

class Product(models.Model):
    user = models.ForeignKey(User, blank=True, null=True, on_delete=models.SET_NULL) 
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)
    
    def get_absolute_url(self): # <-------
        return f"/products/products/{self.slug}/"
    

class DigitalProduct(Product):
    class Meta:
        proxy = True

### src/products/urls.py

In [ ]:
    # path("my-products/<slug:slug>", ProtectedProductDetailView.as_view()),
    path("my-products/create/", ProtectedProductCreateView.as_view()), # <-------
]

## UpdateView y DeleteView

### src/products/views.py

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin
from django.http import HttpResponseRedirect
from django.views.generic import(
    ListView,
    DetailView,RedirectView,
    CreateView,
    UpdateView, # <-------
    DeleteView, # <-------
]

from django.shortcuts import render, get_object_or_404

from .forms import ProductModelForm
from .mixins import TemplateTitleMixin
from .models import Product, DigitalProduct

class ProtectedProductUpdateView(LoginRequiredMixin, UpdateView):
    form_class = ProductModelForm
    template_name = "products/product_detail.html"

    def get_queryset(self):
        return Product.objects.filter(user=self.request.user)

    def get_success_url(self):
        return self.object.get_edit_url()

class ProtectedProductDeleteView(LoginRequiredMixin, DeleteView):
    template_name = "forms_delete.html"

    def get_queryset(self):
        return Product.objects.filter(user=self.request.user)

    def get_success_url(self):
        return "/products/products"

### src/products/urls.py

In [ ]:
from products.views import(
...
    ProtectedProductUpdateView, # <-------
    ProtectedProductDeleteView, # <-------
)

urlpatterns = [
    ...
    path("my-products/<slug:slug>/", ProtectedProductUpdateView.as_view()),
    path("my-products/<slug:slug>/", ProtectedProductDeleteView.as_view()),
]

### src/templates/products/product_detail.html

In [ ]:
{% if title %}
    <h1>{{ title }}</h1>
{% endif %}

 {{ object.pk }}
 {{ object.title }}
 {{ object.slug }}

 {% if form %}
     {% include 'forms.html" with form=form delete_url=object.get_delete_url %}
{% endif %}

### src/products/models.py

In [ ]:
from django.conf import settings
from django.db import models

User = settings.AUTH_USER_MODEL

class Product(models.Model):
    user = models.ForeignKey(User, blank=True, null=True, on_delete=models.SET_NULL)
    title = models.CharField(max_length=255)
    slug = models.SlugField(max_length=255, unique=True)

    def get_absolute_url(self):
        return f"/products/products/{self.slug}"

    def get_edit_url(self):
        return f"/products/my-products/{self.slug}"

    def get_delete_url(self):
        return f"/products/my-products/{self.slug}/delete"

### src/templates/forms.html

In [ ]:
<form method="POST" action=".">
    {% csrf_token %} # indica que el request proviene de una fuente de confianza
    {{ form.as_p }} # indicamos que el form muestre todos los campos
    <input type="submit" value="Guardar"> # botón con etiqueta Guardar
    {% if delete_url %} # 
        <a href="{{ delete_url }}">Eliminar</a>
    {% endif %}
</form>

### src/templates/products/forms_delete.html


In [ ]:
<form method="POST" action="{{ object.get_delete_url}}">
    {% csrf_token %}
    <h1>¿Estás seguro que quieres eliminar "{{ object.title }}"?</h1>
    <input type="submit" value="Confirmar">
</form>